<a href="https://colab.research.google.com/github/MSDS-Capstone-Project/LLM_FineTuning/blob/main/LoRA_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
### Loading the Dataset

try: from datasets import load_dataset
except:
    !pip install datasets
    from datasets import load_dataset


# dataset = load_dataset("nvidia/OpenMathInstruct-2", split = "train_1M")
dataset = load_dataset("commonsense_qa")
# dataset_small = dataset.select(range(10000))
# dataset_split = dataset_small.train_test_split(test_size=0.1, seed=42)
# train_data, eval_data = dataset_split["train"], dataset_split["test"]
train_data, eval_data = dataset["train"], dataset["validation"]

print(train_data)

Dataset({
    features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
    num_rows: 9741
})


In [2]:
eval_data.to_pandas().head()

,id,question,question_concept,choices,answerKey
0,1afa02df02c908a558b4036e80242fac,A revolving door is convenient for two directi...,revolving door,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
1,a7ab086045575bb497933726e4e6ad28,What do people aim to do at work?,people,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
2,b8c0a4703079cf661d7261a60a1bcbff,Where would you find magazines along side many...,magazines,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",B
3,e68fb2448fd74e402aae9982aa76e527,Where are you likely to find a hamburger?,hamburger,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
4,2435de612dd69f2012b9e40d6af4ce38,James was looking for a good place to buy farm...,farmland,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A


In [3]:
### Logging into HuggingFace

try:
  from dotenv import load_dotenv
except:
  !pip install python-dotenv
  from dotenv import load_dotenv

from huggingface_hub import login, whoami
import os

load_dotenv()
login(os.getenv("hugging_face_key2"))

In [1]:
!pip install peft

  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl (127.9 MB)
Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl (207.5 MB)
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 9.3.0.75
    Uninstalling nvidia-cudnn-cu12-9.3.0.75:
      Successfully uninstalled nvidia-cudnn-cu12-9.3.0.75
  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.3.83
    Uninstalling nvidia-cusolver-cu12-11.6.3.83:
      Successfully uninstalled nvidia-cusolver-cu12-11.6.3.83


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
model_name = "gpt2"

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4"
# )

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # device_map="auto",
    # quantization_config=bnb_config,
    # torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

### LoRA Config

from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,  # Rank of the low-rank adaptation - attention heads
    lora_alpha=32, # alpha scaling
    lora_dropout=0.1,
    bias="none",
    target_modules=["c_attn", "c_proj"]  # GPT-2 specific layers
)


peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [5]:
import numpy as np
def format_prompt(example):
    question = example['question']
    choices = example['choices']['text']
    labels = example['choices']['label']
    answer_key = example['answerKey']

    # Find the correct answer text
    answer_index = labels.index(answer_key)
    answer_text = choices[answer_index]

    # Create the prompt
    prompt = f"Q: {question}\nO: {' /// '.join(choices)}\nA: "
    full_text = prompt + " " + answer_text

    # Tokenize full input
    tokenized = tokenizer(
        full_text,
        padding="max_length",
        truncation=True,
        max_length=128,
    )

    # Calculate where the completion starts
    prompt_ids = tokenizer(prompt,
                           truncation=True,
                           max_length=128)["input_ids"]
    completion_start = len(prompt_ids)

    # Mask prompt tokens in the labels
    labels = [-np.inf] * completion_start + tokenized["input_ids"][completion_start:]
    labels += [-np.inf] * (128 - len(labels))  # pad label to match input

    tokenized["labels"] = labels
    return tokenized


tokenized_train = train_data.map(format_prompt, batched=False, remove_columns=train_data.column_names)
tokenized_eval = eval_data.map(format_prompt, batched=False, remove_columns=eval_data.column_names)

tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_eval.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/9741 [00:00<?, ? examples/s]

Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [6]:
tokenized_train.to_pandas().head()

,input_ids,attention_mask,labels
0,"[48, 25, 383, 9388, 1028, 262, 1524, 547, 257,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."
1,"[48, 25, 44067, 2227, 284, 467, 284, 810, 262,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."
2,"[48, 25, 1675, 17276, 257, 442, 11020, 407, 51...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."
3,"[48, 25, 3012, 20347, 290, 584, 12763, 290, 46...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."
4,"[48, 25, 383, 21831, 6807, 422, 262, 1748, 656...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in..."


In [7]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


training_args = TrainingArguments(
    output_dir=f"./IA3_{model_name}",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    eval_strategy="steps",
    eval_steps=100,
    save_steps=1000,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",

    learning_rate=5e-3,

)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
)


trainer.train()



No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
100,3.308800,No log
200,3.133700,No log
300,3.112100,No log
400,3.155500,No log
500,3.128600,No log
600,3.117100,No log
700,3.137500,No log
800,3.108600,No log
900,3.081300,No log
1000,3.079300,No log


TrainOutput(global_step=3654, training_loss=2.848482901370127, metrics={'train_runtime': 609.4037, 'train_samples_per_second': 47.953, 'train_steps_per_second': 5.996, 'total_flos': 1927135929434112.0, 'train_loss': 2.848482901370127, 'epoch': 3.0})

In [8]:
peft_model.save_pretrained(f"./LoRA_{model_name}_final")
tokenizer.save_pretrained(f"./LoRA_{model_name}_final")

('./LoRA_gpt2_final/tokenizer_config.json',
 './LoRA_gpt2_final/special_tokens_map.json',
 './LoRA_gpt2_final/vocab.json',
 './LoRA_gpt2_final/merges.txt',
 './LoRA_gpt2_final/added_tokens.json',
 './LoRA_gpt2_final/tokenizer.json')

In [9]:
from peft import PeftModel, PeftConfig
peft_config = PeftConfig.from_pretrained(f"./LoRA_{model_name}_final")

base_model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path)
model = PeftModel.from_pretrained(base_model, f"./LoRA_{model_name}_final")

tokenizer = AutoTokenizer.from_pretrained(f"./LoRA_{model_name}_final")

In [10]:
from transformers import pipeline
import torch
from tqdm import tqdm

# Use the PEFT model (already trained) and tokenizer
peft_model.eval()
pipe = pipeline("text-generation", model=peft_model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

def format_prompt(example):
    question = example["question"]
    choices = example["choices"]["text"]
    prompt = f"Q: {question}\nO: {' /// '.join(choices)}\nA: "
    return prompt, choices

correct = 0
total = 0

for example in tqdm(eval_data, total=len(eval_data)):
    prompt, choices = format_prompt(example)

    # Generate prediction
    output = pipe(prompt, max_new_tokens=10, do_sample=False, eos_token_id=tokenizer.eos_token_id)[0]["generated_text"]

    # Extract generated answer (after "Answer:")
    generated = output.split("\n")[2].split("A:")[-1].strip().split(" /// ")[0].strip()


    # Get ground truth
    answer_key = example["answerKey"]
    labels = example["choices"]["label"]
    true_answer = example["choices"]["text"][labels.index(answer_key)].strip()

    print("\n".join(output.split("\n")[:3]))
    print(f"\n\nModel: {generated} and Actual Answer: {true_answer}")

    if generated == true_answer:
        correct += 1
    total += 1

accuracy = correct / total
print(f"Accuracy: {accuracy:.2%}")


Device set to use cuda:0
The model 'PeftModel' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM',

Q: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?
O: bank /// library /// department store /// mall /// new york
A:  mall store store store store store store store store store


Model: mall store store store store store store store store store and Actual Answer: bank
Q: What do people aim to do at work?
O: complete job /// learn from each other /// kill animals /// wear hats /// talk to each other
A:  complete job or complete job or complete job or complete


Model: complete job or complete job or complete job or complete and Actual Answer: complete job


  0%|          | 4/1221 [00:00<04:24,  4.61it/s]

Q: Where would you find magazines along side many other printed works?
O: doctor /// bookstore /// market /// train station /// mortuary
A:  bookstore station station station station station station station station station


Model: bookstore station station station station station station station station station and Actual Answer: bookstore
Q: Where are  you likely to find a hamburger?
O: fast food restaurant /// pizza /// ground up dead cows /// mouth /// cow carcus
A:  pizza restaurant restaurant restaurant restaurant restaurant restaurant restaurant restaurant restaurant


Model: pizza restaurant restaurant restaurant restaurant restaurant restaurant restaurant restaurant restaurant and Actual Answer: fast food restaurant


  0%|          | 6/1221 [00:01<03:51,  5.24it/s]

...

Q: If you really wanted a grape, where would you go to get it?
O: winery /// fruit stand /// field /// kitchen /// food
A:  fruit stand stand stand stand stand stand stand stand stand


Model: fruit stand stand stand stand stand stand stand stand stand and Actual Answer: fruit stand
Q: What would you do to a rock when climb up a cliff?
O: grab /// look down /// throw /// falling /// may fall
A:  fall down the cliff /// fall down the cliff ///


Model: fall down the cliff and Actual Answer: grab


100%|█████████▉| 1219/1221 [03:31<00:00,  5.83it/s]

Q: His compressor needed a new hose, where did he go?
O: garden shed /// hardware store /// brothel /// garage /// greenhouse
A:  hardware store /// hardware store /// hardware store


Model: hardware store and Actual Answer: hardware store
Q: The man closed his eyes as the music played, what effect did the music have?
O: coma /// enjoyable /// soothing /// universal /// good or bad
A:  soothing or bad or bad or bad or bad or


Model: soothing or bad or bad or bad or bad or and Actual Answer: soothing


100%|██████████| 1221/1221 [03:31<00:00,  5.78it/s]

Q: Setting up framing, truss and beam are some of the first steps in what?
O: new construction /// warehouse /// driving /// ceiling /// bridge
A:  bridge truss and beam truss and beam tr


Model: bridge truss and beam truss and beam tr and Actual Answer: new construction
Q: What is another name for a disk for storing information?
O: computer store /// computer to store data /// computer hard drive /// cd player /// usb mouse
A:  computer store data to store data to computer hard drive


Model: computer store data to store data to computer hard drive and Actual Answer: computer hard drive
Accuracy: 3.11%


In [11]:
model.eval()
import torch
# prompt = f"Q: {question}\nChoices:\n"
#     for choice in choices:
#         prompt += f"- {choice}\n"
#     prompt += "Answer:"
q = "What do people aim to do at work?"
options = ['complete job', 'learn from each other', 'kill animals', 'wear hats', 'talk to each other']
options2 = ' /// '.join(options)
from IPython.display import display, Math
prompt = f"Q: {q}\nO: {options2}\nA: "
inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)

with torch.no_grad():
    outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=10,
    do_sample=False,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    early_stopping=True,
    num_beams=5,  # Optional: beam search for stronger outputs
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Q: What do people aim to do at work?
O: complete job /// learn from each other /// kill animals /// wear hats /// talk to each other
A:  complete job or complete job or complete job or complete
